## Intein Finder

Source code: https://github.com/omsai/intein_finder

### Project goal

Identify putative inteins in a given genome
by training position weight matrices of known InBase inteins
using PSI Blast or similar.

### Suggestions

- Focus on identifying splicing domains (A, B, F, G) and homing endonucleases (C, D, E, H);
  not necessarily all the annotated domains which have lots of variation.
- Then do a phylogeny of the homing endonucleases and see if they are really vertically inherited
  or see if they horizontally jump back and forth and are the basis of some phylogenies.
  
[![Intein domains](http://www.biocenter.helsinki.fi/bi/iwai/InBase/tools.neb.com/inbase/blocks.gif)](http://www.biocenter.helsinki.fi/bi/iwai/InBase/tools.neb.com/inbase/motifs_splice.html)

In [ ]:
from inbase import INBASE
import pandas as pd
from pprint import pprint

pprint(INBASE.columns.tolist())

## Cleanup motif  annotations

- Remove invalid entries of blank values or dashes.
- Split the numeric location of the protein motif from the motif sequence.

In [ ]:
cols_domain = [col for col in INBASE.columns if 'Block' in col]
INBASE.loc[:, cols_domain].head()

In [ ]:
temp = INBASE.loc[:, cols_domain].stack()
valid = temp.str.match('[A-Z*?/ ]+[0-9]+')
inbase = temp[valid].unstack()
inbase.head()

Split the location numbers from the domain strings.

In [ ]:
domains = pd.DataFrame()

for col in cols_domain:
    block = inbase[col]
    col_new = col.replace(' ', '_')
    block = block.str.extract('(?P<{block}>^[A-Z*?/]+)[ NC]*(?P<{loc}>[0-9]+$)'.format(
            block=col_new, loc='Loc_' + col[-1]), expand=True)
    domains = pd.concat([domains, block], axis=1)

domains.head()

Add our improved columns back into INBASE.

In [ ]:
if len(INBASE.columns) == len(set(cols_domain + INBASE.columns.tolist())):
    INBASE = INBASE.drop(cols_domain, axis=1)
    INBASE = pd.concat([INBASE, domains], axis=1)

import re

cols_splicing = [col for col in domains.columns if re.match('^Block_[A,B,F,G]$', col)]
cols_endonuclease = [col for col in domains.columns if re.match('^Block_[C-E,H]$', col)]
cols_both = cols_splicing + cols_endonuclease
{'Splicing domains': cols_splicing, 'Endonuclease domains': cols_endonuclease}

It looks like the endonuclease H domain is missing from the database.

### Experimental data

- Would it be more accurate to only use experimentally validated inteins to train our PSI-Blast position weight matrix?
- Is 90 inteins enough data?
- Of those 90 inteins, 55 have the splicing domain, 47 have the homing endonuclease domain, and 47 have both.

In [ ]:
INBASE['Intein Class'].unique()

In [ ]:
exp = INBASE[INBASE['Intein Class'] == 'Experimental']
print('%d out of %d inteins (%d%%) were experimentally validated' % (
        len(exp), len(INBASE), len(exp) * 100.0 / len(INBASE)))

In [ ]:
print('Experimentally valid organism domains:')
exp['Domain of Life'].value_counts()

In [ ]:
has_all_splicing_domains = exp[cols_splicing].notna().apply(
    lambda row: all(row[1:len(cols_splicing)]), axis=1)
exp_splicing = exp[has_all_splicing_domains]
print('%d out of %d experimentally validated inteins (%d%%) all have splicing domains' % (
        len(exp_splicing), len(exp), len(exp_splicing) * 100.0 / len(exp)))
exp_splicing['Domain of Life'].value_counts()

In [ ]:
has_all_endonuclease_domains = exp[cols_endonuclease].notna().apply(
    lambda row: all(row[1:len(cols_endonuclease)]), axis=1)
exp_endonuclease = exp[has_all_endonuclease_domains]
print('%d out of %d experimentally validated inteins (%d%%) all have endonuclease domains' % (
        len(exp_endonuclease), len(exp), len(exp_endonuclease) * 100.0 / len(exp)))
exp_endonuclease['Domain of Life'].value_counts()

In [ ]:
has_all_intein_domains = has_all_splicing_domains & has_all_endonuclease_domains
has_all_intein_domains.sum()

In [ ]:
exp.loc[has_all_intein_domains, 'Domain of Life'].value_counts()

In [ ]:
cols_summary = ['Domain of Life', 'Organism Name'] + cols_both
exp.loc[has_all_intein_domains, cols_summary]